## 此文件请在谷歌colab平台上运行

In [3]:
pip install transformers

     |████████████████████████████████| 3.5 MB 12.2 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
     |████████████████████████████████| 6.8 MB 38.3 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 596 kB 48.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader,Dataset
from transformers import BertTokenizer,BertForSequenceClassification
from transformers import AdamW

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data=pd.read_csv('/content/drive/My Drive/data/大样本.csv',names=['index','date','title'],header=0)
data=data.set_index(['index','date'])
data.tail(20)

title
index  date                               
601009 06-25                    为啥我的债没有显示呢
       06-25              说好的重大利好，看看能不能红一下
       06-25                     最后高位出逃机会！
       06-25               宁波银行的主力到南京银行来了。
       06-25         股息0.4元投资债券来估值！预估8元买到！
       06-25                        今年目标13
       06-25     你说对了[赞][赞][赞][献花][献花][献花]
       06-25     各位南行粉丝，请教一个问题，怎么看待周五的无量拉升
       06-25  南京银行本周融资净偿还2002.8万元，居银行板块第十三
       06-25           感觉有大资金运作，今年南京有很好表现。
       06-25   南京银行本周沪股通持股市值增加1.05亿元，居银行板块
       06-25               城商行和农商行净息差达历史低位
       06-25     又分红，又配债，又有打新市值，还不断的创新高，南行
       06-25      我想知道你们有没有给苏宁贷款？如有，具体贷了多少
       06-25                      礼拜一要破新高啊
       06-25                     可转债能赚30％吗
       06-25               银行股分化严重，期待中报收鱼笼
       06-25                    可转债什么时候上市？
       06-25                  有人画的，相信吗？我信。
       06-25     这货在银行里，趋势好，筹码稳定，下周不知道有没有机

In [7]:
tokenizer=BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
model=BertForSequenceClassification.from_pretrained('hfl/chinese-roberta-wwm-ext',num_labels=2)

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [8]:
text=data['title'].tolist()
device=torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [15]:
output=pd.DataFrame()
for i in range(1,8000):
    temp_text=text[(i-1)*10:i*10]
    batch=tokenizer(temp_text,
    padding=True,
    truncation=True,
    max_length=50,
    return_tensors='pt')#一定要返回pt
    batch.to(device)
    res=model(**batch)
    activation=torch.nn.functional.softmax(res.logits,dim=1)
    temp_output_df=pd.DataFrame()
    temp_output_df['score']=activation[:,1].tolist()
    output=pd.concat([output,temp_output_df])

In [16]:
temp_data=data.reset_index()
output=output.reset_index(drop=True)

temp_data=pd.concat([temp_data,output],axis=1)
temp_data=temp_data.set_index(['index','date'])

In [20]:
temp_data

title     score
index  date                                            
601398 02-12         指数上涨≠赚钱效应，这两个板块虎年一定会出大牛股  0.236162
       02-12                               转发  0.191862
       02-12                               转发  0.191862
       02-12       中国政府网2月11日消息，国务院近期印发《“十四五”  0.232632
       02-12  $工商银行(SH601398)$这个账户的工行赚了5.25%！  0.172973
...                                       ...       ...
601009 06-25                        可转债能赚30％吗  0.192436
       06-25                  银行股分化严重，期待中报收鱼笼  0.228757
       06-25                       可转债什么时候上市？  0.252376
       06-25                     有人画的，相信吗？我信。  0.221031
       06-25        这货在银行里，趋势好，筹码稳定，下周不知道有没有机  0.177981

[79990 rows x 2 columns]

In [26]:
drive.mount('/content/drive')
temp_data.to_csv('/content/drive/My Drive/data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
